# Поврзи 4

## Лабораториска вежба 5

Имате задача да создадете вештачка интелигенција која ќе умее да ја игра играта [Поврзи 4](https://en.wikipedia.org/wiki/Game_of_the_Amazons).

Во функцијата `get_move_AI()` имплементирајте минимакс алгоритам кој ќе истражува до одредена длабочина. Кога ќе стигне до однапред зададената длабочина во одредена гранка, повикајте ја функцијата `evaluate_state()` која треба да процени кој играч е во подобра позиција, па да го врати најдобриот потег.

In [1]:
from time import sleep
import random
from copy import deepcopy
import threading
import queue
from plotly import graph_objects as go
import ipywidgets as ipw
from IPython import display

In [2]:
E = EMPTY = '·'
R = 'R'
Y = 'Y'
inf = float('inf')
MAX_DEPTH = 3

In [3]:
class Game:
    def __init__(
        self, player_1, player_2, state=None, time_to_play=10, n_rows=6, n_cols=7
    ):
        self.n_rows, self.n_cols = n_rows, n_cols
        if state is None:
            state = self.empty_state()
        self.state_original = state
        self.time_to_play = time_to_play
        self.player_1 = {**player_1, 'symbol': 'R'}
        self.player_2 = {**player_2, 'symbol': 'Y'}
        self.symbols_fig = {'R': 'circle', 'Y': 'circle', EMPTY: 'circle-open'}
        self.colors_fig = {'R': 'Tomato', 'Y': 'Gold', EMPTY: 'LightSkyBlue'}
        self.scores = {'R': inf, 'Y': -inf}
        self.create_ui()
        self.restart()
        self.initial_turn()
        
    def empty_state(self):
        return [[EMPTY for _ in range(self.n_cols)] for _ in range(self.n_rows)]
    
    def _ipython_display_(self):
        display.display(self.ui)
        
    def restart(self, *args):
        self.state = deepcopy(self.state_original)
        self.active_player = self.player_1
        self.update_score('На ред е', self.active_player)
        self.winner = 'No result yet'
        self.redraw_symbols()
        self.output.clear_output(wait=True)
        
    def ck_bt_restart(self, bt_restart):
        self.restart()
        self.initial_turn()

    def initial_turn(self):
        if 'human' not in [self.player_1['type'], self.player_2['type']]:
            while self.winner == 'No result yet':
                self.ai_move()
        elif self.active_player['type'] == 'AI':
            self.ai_move()
        
    def create_ui(self):
        self.dashboard = ipw.HTML(description='Статус:', value='')
        self.bt_restart = ipw.Button(description='Одново')
        self.bt_restart.on_click(self.ck_bt_restart)
        self.fig = self.create_fig()

        hbox = ipw.HBox([self.bt_restart, self.dashboard])
        self.output = ipw.Output()
        self.ui = ipw.VBox([hbox, self.fig, self.output])
        
    def create_fig(self):
        fig = go.FigureWidget()
        x = [x for y in range(self.n_rows) for x in range(self.n_cols)]
        y = [y for y in range(self.n_rows) for x in range(self.n_cols)]
        fig.add_scatter(
            x=x, y=y, mode='markers', marker_size=48,
            marker_symbol='circle-open', marker_color='LightSkyBlue',
            marker_line_width=6, marker_line_color='LightSkyBlue')
        fig.data[0].on_click(self.human_move)
        fig.update_xaxes(
            range=[-0.5, self.n_cols - 0.5], dtick=1, title='x', side='top')
        fig.update_yaxes(
            range=[-0.5, self.n_rows - 0.5], dtick=1, title='y',
            autorange='reversed')
        fig.update_layout(width=700, height=600, showlegend=False)
        return fig

    def get_symbols_for_draw(self):
        return [self.symbols_fig[v] for row in self.state for v in row]
    
    def get_colors_for_draw(self):
        return [self.colors_fig[v] for row in self.state for v in row]

    def its_valid_response(self, move):
        if move == 'did not respond':
            return False, 'Player did not respond in time.'
        if move is None:
            return False, 'Player\'s response is None.'
        if type(move) != int:
            return False, 'Player\'s response is not an int.'
        if move not in range(0, self.n_cols):
            return False, f'Player\'s response ({move}) is not in range(0, n_cols).'
        x, y = move, self.drop_y(self.state, move)
        if y not in range(0, self.n_cols):
            return False, 'Player\'s plays over full column.'
        return True, ''

    def get_random_move(self):
        all_x = list(range(self.n_cols))
        random.shuffle(all_x)
        for x in all_x:
            y = self.drop_y(self.state, x)
            if y != -1:
                return y, x
    
    def ai_move(self):
        move = self.wait_for_player()
        valid, reason = self.its_valid_response(move)
        if valid:
            x, y = move, self.drop_y(self.state, move)
        else:
            name = self.active_player['name']
            msg = f'{name} - Invalid move: {reason}'
            msg += ' A random move will be played.'
            self.print_to_dash(msg)
            y, x = self.get_random_move()
        self.state[y][x] = self.active_player['symbol']
        self.after_turn((y, x))

    def print_to_dash(self, msg):
        with self.output:
            display.display(msg)

    def wait_for_player(self):
        move = 'did not respond'
        q_result = queue.Queue()
        get_move = self.active_player['get_move']
        worker = lambda: q_result.put(get_move(deepcopy(self.state), self.active_player))
        thread = threading.Thread(target=worker)
        thread.start()
        thread.join(timeout=self.time_to_play)
        if thread.is_alive():
            pass
        else:
            move = q_result.get()
        return move

    def drop_y(self, state, x):
        return sum([1 if row[x] == EMPTY else 0 for row in state]) - 1

    def human_move(self, trace, points, selector):
        if self.winner != 'No result yet':
            return
        x, y = points.xs[0], points.ys[0]
        y = self.drop_y(self.state, x)
        if y not in range(0, self.n_rows):
            return
        self.state[y][x] = self.active_player['symbol']
        
        self.after_turn((y, x))
        if self.active_player['type'] == 'AI':
            if self.winner == 'No result yet':
                self.ai_move()

    def redraw_symbols(self):
        self.fig.data[0].marker.symbol = self.get_symbols_for_draw()
        self.fig.data[0].marker.color = self.get_colors_for_draw()

    def flip_active_player(self):
        if self.active_player == self.player_2:
            return self.player_1
        return self.player_2

    def after_turn(self, turn):
        self.redraw_symbols()
        name = self.active_player['name']
        y, x = turn
        self.print_to_dash(f'{name} plays (column, row) ({x}, {y}).')
        self.winner = self.check_victory(self.state)
        if self.winner != 'No result yet':
            if self.winner == 'Victory':
                self.update_score('Победник е', self.active_player)
            if self.winner == 'Draw':
                self.update_score('Нерешено')
            return
        # flip
        self.active_player = self.flip_active_player()
        self.update_score('На ред е', self.active_player)
        
    def update_score(self, message, player=None):
        name = '' if player is None else player['name']
        self.dashboard.value = f'{message} <b> {name} </b>.'

    def check_victory(self, state):
        def check_line(line):
            for i in range(len(line) - 3):
                if all(cell == line[i] and cell != EMPTY for cell in line[i:i + 4]):
                    return True
            return False

        def check_columns(state):
            for col in range(len(state[0])):
                column = [state[row][col] for row in range(len(state))]
                if check_line(column):
                    return True
            return False

        def check_rows(state):
            for row in state:
                if check_line(row):
                    return True
            return False

        def check_diagonals(state):
            for i in range(len(state) - 3):
                for j in range(len(state[0]) - 3):
                    diagonal = [state[i + k][j + k] for k in range(4)]
                    if check_line(diagonal):
                        return True

                    diagonal = [state[i + 3 - k][j + k] for k in range(4)]
                    if check_line(diagonal):
                        return True
            return False

        def check_draw(state):
            return all(cell != EMPTY for row in state for cell in row)

        if check_columns(state) or check_rows(state) or check_diagonals(state):
            return 'Victory'
        elif check_draw(state):
            return 'Draw'
        else:
            return 'No result yet'

In [4]:
def expand_state(state, player):
    for x in range(7):
        y = sum([1 if row[x] == EMPTY else 0 for row in state]) - 1
        if y == -1:
            continue
        new_state = list(map(list, state))
        new_state[y][x] = 'R' if player == 'MAX' else 'Y'
        yield new_state, x

In [5]:
def evaluate_state(state, player):
    """ Проценува колкава е предноста за играчот MAX (негативно е предност за MIN) """
    " Вашиот код тука "
    # избришете ги линиите каде се генерираат случајни вредности
    import random
    return random.random() * 2 - 1

In [6]:
def get_move_AI(state, player):
    MM = player['MM']
    result, move = minimax(state, MM)
    return move

def minimax(node, player, alpha=-inf, beta=inf, depth=0):
    best_value = inf if player == 'MIN' else -inf
    best_move = None
    for child, move in expand_state(node, player):
        other_player = 'MIN' if player == 'MAX' else 'MAX'
        if depth < MAX_DEPTH:
            result, _ = minimax(child, other_player, alpha, beta, depth+1)
        else:
            result = evaluate_state(child, other_player)
        if player == 'MIN':
            if result <= alpha:
                return result, best_move
            if result < beta:
                beta = result
            if result < best_value:
                best_value = result
                best_move = move
        elif player == 'MAX':
            if result >= beta:
                return result, best_move
            if result > alpha:
                alpha = result
            if result > best_value:
                best_value = result
                best_move = move
    return best_value, best_move

    
player_1 = {'name': 'Црвен','MM': 'MAX', 'type': 'AI', 'get_move': get_move_AI}
player_2 = {'name': 'Жолт','MM': 'MIN', 'type': 'human'}
Game(player_1, player_2)